In [ ]:
# from langchain_huggingface import HuggingFaceEndpoint
# from langchain.prompts import PromptTemplate

# prompt = PromptTemplate.from_template("What is the meaning of {word}")

# llm = HuggingFaceEndpoint(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.1",
#     max_new_tokens=250,
# )

# chain = prompt | llm

# chain.invoke({"word": "potato"})

# 아래 코드는 정상 동작함.
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    max_new_tokens=250,
)

prompt = PromptTemplate.from_template("What is the meaning of {word}")
chain = prompt | llm
print(chain.invoke({"word": "potato"}))

d:\Programming\NomadCoder\FullStackGPT\gpt-project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


StopIteration: 